In [ ]:
import requests as req
import pandas as pd
import sqlite3 as sql
import requests
from functools import reduce
import numpy as np
import os.path
from os import path
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
from sklearn.linear_model import LinearRegression

In [ ]:
sqlite_file = 'salaries_data.sql'

In [ ]:
db = sql.connect(":memory:")

In [ ]:
sql_file = open(sqlite_file)
sql_as_string = sql_file. read()
db. executescript(sql_as_string)

## Data  Manipulation

In [ ]:
salary_query= """
SELECT 
 firstname, lastname, department, salary, position
FROM Salaries
"""

In [ ]:
salaries = pd.read_sql(salary_query, db)

In [ ]:
salaries.head()

In [ ]:
def count_nonempty(d): 
    total = 0
    for k in d.keys():
        if (d.get(k) != None):
            total += 1
    print("the non empty total", total)

In [ ]:
headers = {
  'Accept': 'application/json'
}

In [ ]:
def fetch_profs(): 


    profs = dict()
    matched_profs = dict()

    for i in range(0, 10000, 1000):
        r = requests.get('https://api.planetterp.com/v1/professors?reviews=true', params={
          'limit': '1000',
          'offset': i,
        }, headers = headers)

        for e in r.json():
            profs[e.get("slug")] = e
            matched_profs[e.get("slug")] = None

    
    print(len(profs.values()))
    return (profs, matched_profs)

In [ ]:
def find_closest_match(name, matches):
    firstname = name.split(" ")[0]
    ret = [ m  for m in matches if m[0].find(firstname) >= 0 ]
    if(len(ret) > 0):
        return ret[0]
    return None
def calc_avg_review(revs):
    length = len(revs)
    if(length == 0):
        return np.nan
    total = 0
    for r in revs:
        total += r.get("rating")
    return total / length
        

In [ ]:
def get_department(dep, profs, matched_profs):
    matches = dict()

    for prof in profs.values():
        
        slug = prof.get("slug")
        name = prof.get("name")
        courses = prof.get("courses")
        avg_rating = calc_avg_review(prof.get("reviews"))
        
        if(matched_profs.get(slug) != None):
            continue

        lastname = name.split(" ")[-1]

        salary_query= f"""
            SELECT 
            firstname, 
            lastname,
            department,
            salary,
            position
            FROM Salaries
            WHERE lastname like "{lastname}"
            AND department = "{dep}"
        """
        rows = list(db.execute(salary_query))
        matches[lastname] = (len(rows), rows, name, slug, courses, avg_rating)
        

    for match in matches.keys():
        (length, v, name, slug, courses, avg_rating) = matches.get(match)
        if(length > 0):
           
            value = find_closest_match(name, v)
         
            if(value != None):
                
                matched_profs[slug] = (
                    value[0],
                    value[1],
                    value[2],
                    value[3],
                    value[4],
                    avg_rating,
                    slug,
                    name,
                    "-".join(courses),
                    len(courses)
                )
                

    return

In [ ]:
data = []

In [ ]:
def fetch_matched_salaries():
    if(path.isfile("matched_salaries.csv")):
        return pd.read_csv("matched_salaries.csv")
    
    
    (profs, matched_profs) = fetch_profs()
    
    arr = list(set(salaries["department"]))
    for dep in arr:   
        get_department(dep, profs, matched_profs)
        
    data = [ list(v)  for v in matched_profs.values() if v != None]
    matched_salaries = pd.DataFrame(data, columns=['firstname', 'lastname', 'department', 'salary', "position", "avg_rating", 'slug', "planetTerpName", "courses", "num_courses"])
    matched_salaries.to_csv(path_or_buf="matched_salaries.csv", index=False)
    return matched_salaries


In [ ]:
matched_salaries = fetch_matched_salaries()
matched_salaries = matched_salaries[matched_salaries['position'].str.contains("Professor") | matched_salaries['position'].str.contains("Lecturer")]
matched_salaries

In [ ]:
print(matched_salaries.groupby(by='department').mean())

In [ ]:
def fetch_reviewsData(_data, ids):
    if(path.isfile("reviews.csv")):
        return pd.read_csv("reviews.csv")
        
    reviews_data = []
    i = 0
    for v in _data:

        r = requests.get('https://api.planetterp.com/v1/professor', params={
          'name': v,  'reviews': 'true'
        }, headers = headers).json()


        slug = r.get("slug")
        revs = r.get("reviews")
        if(revs == None):
            continue

        for rev in revs:  

            reviews_data.append([ 
                slug,
                rev.get("rating"),  
                rev.get("expected_grade"), 
                rev.get("course"),
                rev.get("review"),
                ids[i]
            ])
            
        i += 1;
    reviews_df = pd.DataFrame(reviews_data, columns=['slug', 'rating', 'expected_grade', 'course', 'review', "prof_df_id"])
    reviews_df.to_csv(path_or_buf="reviews.csv", index=False)
    return  reviews_df



In [ ]:
reviews_df = fetch_reviewsData(list(matched_salaries["planetTerpName"]), matched_salaries.index.values)
reviews_df.head()

In [ ]:
# word vector classifications - gensim

# word cloud with reviews (bag of words)
    # words that describe popular profesors
    
# scatter plot of professor salaries vs ratings
# scatter plot of depart salaries vs ratings

# consider the title of the professors which may be a reason for increased salary


## EDA

In [ ]:
dept_avg = matched_salaries.groupby("department").mean().sort_values(by=["salary"], ascending=False)
dept_avg
dept_avg = dept_avg.dropna()

In [ ]:
plt.scatter(dept_avg['salary'], dept_avg['avg_rating'])
plt.xlabel('salary')
plt.ylabel('Average Rating')

m,b = np.polyfit(dept_avg['salary'], dept_avg['avg_rating'], 1)
print(m)
print(b)

plt.plot(dept_avg['salary'], m * dept_avg['salary'] + b)

plt.show()


In [ ]:
prof_salary = matched_salaries.dropna(subset = ['avg_rating'])

prof_salary = prof_salary[prof_salary['position'].str.contains("Professor") | prof_salary['position'].str.contains("Lecturer")]

plt.scatter(prof_salary['salary'], prof_salary['avg_rating'])
plt.xlabel('salary')
plt.ylabel('Average Rating')

m,b = np.polyfit(prof_salary['salary'], prof_salary['avg_rating'], 1)
print(m)
print(b)

plt.plot(prof_salary['salary'], m * prof_salary['salary'] + b, color = 'red')

plt.show()

In [ ]:
prof_salary = matched_salaries.dropna(subset = ['avg_rating'])


# print(prof_salary)

plt.scatter(prof_salary['num_courses'], prof_salary['salary'])
plt.xlabel('num_courses')
plt.ylabel('salary salary')

m,b = np.polyfit(prof_salary['num_courses'], prof_salary['salary'], 1)
# print(m)
# print(b)

plt.plot(prof_salary['num_courses'], m * prof_salary['num_courses'] + b, color = 'red')

plt.show()

In [ ]:
matched_salaries.corr()

In [ ]:
plt.subplots(figsize = (10,4))
g = sns.heatmap(matched_salaries.corr(), annot = True, cmap = "YlGnBu", fmt = '.1g', linewidths=.5,)

g.set_title("Correlation Matrix between Columns")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
dept_features = pd.get_dummies(matched_salaries['department'], drop_first = True, prefix = 'dept')
# print(dept_features)

std_features = matched_salaries[['avg_rating', 'salary', 'num_courses']]

#combine our data
comb_features = pd.concat([std_features, dept_features], axis = 1)
# for i in filter(lambda i : i!= 'AGNR', matched_salaries['department'].unique()):
#     comb_features['salary*' + i] = comb_features['dept_' + i]*comb_features['salary']


In [ ]:
plt.subplots(figsize = (20,11))
g = sns.heatmap(comb_features.corr(), annot = True, cmap = "YlGnBu", fmt = '.1g', linewidths=.5,)

g.set_title("Correlation Matrix between Columns")
g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')

going back


In [ ]:
# def convert_to_num_rep(s):
#     if(pd.isnull(s)):
#         return 0
#     nums = map(lambda c: str(ord(c)) , list(s))
#     return int("".join(list(nums)))

def get_salary(x): 
    return matched_salaries.loc[x.prof_df_id]["salary"]
def get_department(x):
    return matched_salaries.loc[x.prof_df_id]["department"]
    
def map_grades(g): 
    if(pd.isnull(g)):
        return np.nan
    
    d = {'A': 90, "B": 80, "C": 70, "D": 60, "F": 50}
    match = re.search('[a-z|A-Z]', g, re.IGNORECASE)
    if(match == None):
        return np.nan
    val = d.get(match.group(0))
    return val if val != None else np.nan

# reviews_df['course_num'] = reviews_df.apply(lambda x: convert_to_num_rep(x.course), axis=1)
reviews_df['grade_num'] = reviews_df.apply(lambda x: map_grades(x.expected_grade), axis=1)
reviews_df['department'] = reviews_df.apply(lambda x: get_department(x), axis=1)
reviews_df['salary'] = reviews_df.apply(lambda x: get_salary(x), axis=1)
reviews_df.head()

## Hypothesis Testing/Machine Learning

In [ ]:
dept_features = pd.get_dummies(matched_salaries['department'], drop_first = True, prefix = 'dept')
std_features = matched_salaries[['num_courses', 'avg_rating', 'salary']]

comb_features = pd.concat([std_features, dept_features], axis = 1)

comb_features = comb_features.dropna()
for i in filter(lambda i : i!= 'AGNR', matched_salaries['department'].unique()):
    comb_features['num_course*' + i] = comb_features['dept_' + i]*comb_features['num_courses']
    comb_features['rating*' + i] = comb_features['dept_' + i]*comb_features['avg_rating']
    
comb_features

In [ ]:
#seperate our data
X = comb_features.drop('salary', axis = 1)
y = comb_features['salary']
reg = LinearRegression()
reg.fit(X,y)
print("Coefficient of each column: " + str(reg.coef_))
print("Y-intercept: " + str(reg.intercept_))

In [ ]:
print(reg.predict([X.loc[4]]))

In [ ]:
predictions = []
for i, row in matched_salaries.iterrows():
    if i in comb_features.index:
        predictions.append(round(reg.predict([X.loc[i]])[0],2))
        
    else :
        predictions.append(np.nan)
matched_salaries['predicted_salary'] = predictions
matched_salaries.head(50)

In [ ]:
missing_grades = reviews_df[pd.isnull(reviews_df["grade_num"])]
missing_grades

In [ ]:
non_missing_grades = reviews_df.dropna()
non_missing_grades

In [ ]:
partition = math.floor(len(non_missing_grades.index) / 3) * 2
train_sample = non_missing_grades.iloc[0:partition, 0:]
test_sample =non_missing_grades.iloc[partition:, 0:]

In [ ]:
test_sample
train_sample

In [ ]:
train_sample